In [1]:
from ipynb.fs.full.feature_engineering_tree import X_Transformer
from ipynb.fs.full.feature_engineering_tree import TargetNormalizedRegressor
import pandas as pd
from ipynb.fs.full.feature_engineering_tree import DataLoader
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import make_scorer
import numpy as np
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
import numpy.ma as ma
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import skew
from scipy.stats import boxcox_normmax
from scipy.special import boxcox1p
from mlxtend.evaluate import paired_ttest_5x2cv
import seaborn as sns

In [2]:
def rmsle(y_true, y_pred, sample_weight=None, multioutput='uniform_average'):
    return np.sqrt(mean_squared_log_error(y_true, y_pred, sample_weight, multioutput))
    
#I just want to make rmse positive
flipped_score = make_scorer(rmsle, greater_is_better=True)

In [3]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5),scoring=None):
   
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes,scoring = scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [4]:
data_loader = DataLoader("train_cluster.csv","train_cluster.csv")

/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3538: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


In [5]:
X_train = data_loader.getX_train()
Y_train = data_loader.getY_train()

In [6]:
x_transformer = X_Transformer()

/home/victor/anaconda3/lib/python3.6/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [7]:
X_train = x_transformer.fit_transform(X_train,Y_train)
kf = KFold(n_splits=10,shuffle = True,random_state = 42)

/home/victor/anaconda3/lib/python3.6/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
/home/victor/anaconda3/lib/python3.6/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
/home/victor/anaconda3/lib/python3.6/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [8]:
X_train

,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_Nan,BsmtQual_TA,Alley_Grvl,Alley_Nan,Alley_Pave,BsmtCond_Fa,BsmtCond_Gd,...,YrBltAndRemod,TotalSF,Total_sqr_footage,Total_Bathrooms,Total_porch_sf,haspool,has2ndfloor,hasgarage,hasbsmt,hasfireplace
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,4006.0,27.494040,10.670502,3.5,20.313249,0.0,1.0,1.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3952.0,27.354273,10.533228,2.5,58.238312,0.0,0.0,1.0,1.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,4003.0,27.948239,10.558910,3.5,15.750658,0.0,1.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,3885.0,27.182263,10.268004,2.0,59.382673,0.0,1.0,1.0,1.0,1.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,4000.0,29.822291,10.975078,3.5,55.387278,0.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3999.0,27.605971,9.983311,2.5,15.231499,0.0,1.0,1.0,1.0,1.0
1445,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3966.0,30.543592,11.083868,3.0,64.571436,0.0,0.0,1.0,1.0,1.0
1446,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,3947.0,30.222486,10.815014,2.0,20.087052,0.0,1.0,1.0,1.0,1.0
1447,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,3946.0,26.050854,10.464165,2.0,79.258120,0.0,0.0,1.0,1.0,0.0


In [ ]:
#cross validation and predicted vs measured cross validation
gbr_t = TargetNormalizedRegressor(GradientBoostingRegressor(n_estimators=3000, learning_rate=0.01,
                                   max_depth=3, max_features='sqrt',
                                   loss='huber', random_state =42,subsample = 0.7))
scores = cross_val_score(gbr_t, X_train, Y_train, cv=kf,n_jobs=-1,scoring = flipped_score)
print("Rmse log average: %0.5f (+/- %0.5f)" % (scores.mean(), scores.std() * 2))
print("Rmse log median: %0.5f" % (np.median(scores)))
predicted = cross_val_predict(gbr_t, X_train, Y_train, cv=kf,n_jobs=-1)
print(scores)
fig, ax = plt.subplots()
ax.scatter(Y_train, predicted, edgecolors=(0, 0, 0))
ax.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

Rmse log average: 0.10259 (+/- 0.01288)
Rmse log median: 0.10010


In [ ]:
#learning curve of best model
plot_learning_curve(gbr_t, "Title", X_train, Y_train, None, cv=kf, n_jobs=-1,scoring = flipped_score)